In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-pioe62_o
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-pioe62_o
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=9c042482803bf03117874d3f45bc4ec4350d69c1176a50951a15900b1e3f120f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1suoejl2/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [38]:
%%cuda --name LDF.cu

#include "/content/drive/MyDrive/graph/coloring.h"
#include "/content/drive/MyDrive/graph/graph_d.h"

#define THREADxBLOCK 128

using namespace std;

__global__ void findIS (Coloring* col, GraphStruct *str, uint* weigths, bool* candidateNodes) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
  for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
    uint neighDeg = str->cumDegs[neighID + 1] - str->cumDegs[neighID];
		if (col->coloring[neighID] == 0 &&
				((deg < neighDeg) ||
				((deg == neighDeg) && (weigths[idx] < weigths[neighID])))) {
			candidate = false;
		}
	}


 if(candidate){
		candidateNodes[idx] = true;
  }

}

__global__ void colorer (Coloring* col, GraphStruct *str, bool* candidateNodes) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

  if(candidateNodes[idx]){
      bool* forbidden;
			uint n = str->nodeSize;
			cudaMalloc((void**) &forbidden, n * sizeof(bool));
			memset(forbidden, false, n);

			for (uint j = 0; j < deg; j++) {
					uint neighID = str->neighs[offset + j];
					forbidden[col->coloring[neighID]] = true;
			}

			for(uint i = 0; i <= n; i++){
				if(forbidden[i] == false){
					col->coloring[idx] = i;
					free(forbidden);
					return;
				}
			}
		free(forbidden);
	}

	else
	 		col->uncoloredNodes = true;

}

Coloring* graphColoring(GraphStruct *str) {
  Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

  CHECK(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	uint* weigths;
  curandState_t* states;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMalloc((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;

	init <<< blocks, threads >>> (seed, states, weigths, n);

	bool* candidateNodes;
	cudaMalloc((void**) &candidateNodes, n * sizeof(bool));
	cudaMemset(candidateNodes, false, n);

  col->numOfColors = 0;
	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
    col->numOfColors++;
		findIS <<< blocks, threads >>> (col, str, weigths, candidateNodes);
		colorer <<< blocks, threads >>> (col, str, candidateNodes);
		cudaDeviceSynchronize();
	}

	cudaFree(states);
	return col;
}

__global__ void init (uint seed, curandState_t* states, uint* weigths, uint n) {
		uint idx  =threadIdx.x + blockDim.x * blockIdx.x;

		if(idx > n){
			return;
		}

		weigths[idx] = idx;
}


'File written in /content/src/LDF.cu'

In [4]:
%%cuda --name test_LDF.cu

#include "/content/drive/MyDrive/graph/coloring.h"


int main(void) {
	unsigned int n = 1000;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

	srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

	cudaEventRecord(stop);
  cudaEventSynchronize(stop);

	//Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	//printColoring(col, str, 1);
 printf("Coloratura trovata: ");
	for(int i = 0; i < str->nodeSize; i++){
			printf("%d ", col->coloring[i]);
	}

	return EXIT_SUCCESS;
}

'File written in /content/src/test_LDF.cu'

In [39]:
!nvcc -dc src/test_LDF.cu /content/src/LDF.cu /content/drive/MyDrive/graph/graph.cpp /content/drive/MyDrive/graph/graph_d.cu
!nvcc test_LDF.o LDF.o graph.o graph_d.o -o test_LDF
!./test_LDF

318.668793 ms
Coloratura trovata: 10 62 90 102 97 9 64 5 61 118 37 26 36 87 55 111 48 4 16 107 32 101 34 71 86 14 66 69 113 111 63 59 68 27 85 48 33 103 106 12 113 83 64 84 106 27 14 72 23 110 4 76 78 96 109 60 17 103 99 92 3 102 40 58 113 35 117 18 109 89 64 62 78 44 41 53 63 34 87 75 88 22 4 89 94 60 14 97 71 14 101 96 110 21 85 21 82 68 42 86 10 30 29 27 106 20 3 33 101 31 23 62 19 61 55 75 2 98 35 50 5 105 36 27 93 27 24 61 87 49 112 41 20 49 55 83 60 11 12 113 122 6 26 8 92 3 95 6 43 69 105 10 118 30 24 26 51 19 100 49 47 83 45 89 25 32 56 74 31 28 19 44 57 81 43 69 39 9 28 39 112 13 88 3 63 44 77 41 45 117 21 20 59 76 34 27 39 14 95 59 38 43 28 42 46 41 95 21 0 9 61 25 112 108 113 90 9 46 111 81 122 61 43 63 18 2 56 39 18 42 116 28 32 79 66 48 29 99 37 121 55 116 98 76 52 115 45 47 70 40 79 30 103 15 55 28 89 14 51 14 33 80 107 70 54 40 2 32 28 6 7 80 64 74 72 27 104 37 69 53 73 18 34 32 14 92 16 72 18 22 102 0 86 23 82 57 35 56 11 19 71 52 43 58 61 22 3 1 104 59 75 94 103 31 2 1